In [1]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
from transformers import ResNetForImageClassification
from PIL import Image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from torch.nn import Sequential as SequentialTorch, Flatten as FlattenTorch
from sklearn.svm import NuSVC, SVC
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import ast

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout

C:\Users\Prasad\AppData\Roaming\Python\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Prasad\AppData\Roaming\Python\Python310\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model_checkpoint = 'microsoft/resnet-50'
model = ResNetForImageClassification.from_pretrained(model_checkpoint)

model.classifier = SequentialTorch(
    FlattenTorch(start_dim=1, end_dim=-1)
)

In [3]:
orb = cv.ORB_create(64)

In [4]:
folder_path = "./test_data"

In [5]:
features = []
labels = []
handcrafted_features = []
deep_features = []

In [6]:
def extract_label_from_image_name(image_name):
    # Assuming the label is encoded in the last 9 characters
    label = image_name[-14:-5]
    return label

In [7]:
print(len(os.listdir(folder_path)))

105


In [8]:
for image_name in os.listdir(folder_path):
    # Assuming image names have a code that corresponds to the label
    label = extract_label_from_image_name(image_name)

    # Load and preprocess the image
    image_path = os.path.join(folder_path, image_name)
    img = cv.imread(image_path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # ORB feature extraction
    kp = orb.detect(img, None)
    if kp:
        kp, des = orb.compute(img, kp)
        handcrafted_features.append(des.flatten())

        # Check if descriptors are not None and not empty
        if des is not None and des.size > 0:
            # ResNet-50 feature extraction
            image = Image.open(image_path).resize((224, 224))
            if image.mode != 'RGB':
                image = image.convert('RGB')
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image = transform(image)
            inputs = image.unsqueeze(0)
            outputs = model(inputs)
            logits = outputs.logits
            resnet_features = logits.detach().numpy().flatten()
            deep_features.append(resnet_features)

            # Check if resnet_features is not empty
            if resnet_features.size > 0:
                # Combine features
                # combined_features = np.stack((resnet_features, des.flatten()), axis=1)

                # Append to lists
                # features.append(combined_features)
                labels.append(label)

In [9]:
for i in range(len(handcrafted_features)):
    if type(handcrafted_features[i]) != list:
        handcrafted_features[i] = handcrafted_features[i].tolist()
    if len(handcrafted_features[i]) != 2048:
        print(len(handcrafted_features[i]))
        handcrafted_features[i] = handcrafted_features[i] + [0]*(2048 - len(handcrafted_features[i]))
        
print('loop done')
        
for i in range(len(handcrafted_features)):
    if len(handcrafted_features[i]) != 2048:
        print(len(handcrafted_features[i]))

1248
352
512
loop done


In [10]:
handcrafted_features = np.array(handcrafted_features)
deep_features = np.array(deep_features)

In [11]:
combined_features = np.stack((deep_features, handcrafted_features), axis=1)

In [12]:
file_path = './LOC_synset_mapping.txt'

with open(file_path, 'r') as file:
    class_to_name_dict = {}
    for line in file:
        line = line.strip()
        if line:
            parts = line.split(' ', 1)
            class_label = parts[0]
            class_name = parts[1]
            
            class_to_name_dict[class_label] = class_name

In [13]:
file_path = './imagenet1000_clsidx_to_labels.txt'

with open(file_path, 'r') as file:
    content = file.read()
    extracted_dict = ast.literal_eval(content)

class_name_to_numeric_class_dict = {value: key for key, value in extracted_dict.items()}

In [14]:
mapped_labels = []

for i in range(len(labels)):
    class_name_for_label = class_to_name_dict[labels[i]]
    numeric_label = class_name_to_numeric_class_dict[class_name_for_label]
    mapped_labels.append(numeric_label)
    
print(mapped_labels)

[65, 970, 230, 809, 516, 57, 334, 415, 674, 332, 109, 286, 370, 757, 595, 147, 108, 23, 478, 517, 334, 173, 948, 727, 23, 846, 270, 167, 55, 858, 324, 573, 150, 981, 586, 887, 32, 398, 777, 74, 516, 756, 129, 198, 256, 725, 565, 167, 717, 394, 92, 29, 844, 591, 358, 468, 259, 994, 872, 588, 474, 183, 107, 46, 842, 390, 101, 887, 870, 841, 467, 149, 21, 476, 80, 424, 159, 275, 175, 461, 970, 160, 788, 58, 479, 498, 369, 28, 487, 50, 270, 383, 366, 780, 373, 705, 330, 142, 949, 349, 473, 159, 872, 878, 201]


In [19]:
mapped_labels = np.array(mapped_labels)

X_train, X_test, y_train, y_test = train_test_split(handcrafted_features, mapped_labels, test_size=0.1, random_state=100)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(94, 2048) (94,) (11, 2048) (11,)


In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()

lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [22]:
transformed = lda.transform(X_train)
svc = SVC()
svc.fit(transformed, y_train)

SVC()

In [24]:
transformed_test = lda.transform(X_test)
preds = svc.predict(transformed_test)
print(f'Accuracy is : {accuracy_score(y_test, preds)}')

Accuracy is : 0.0
